In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import pyplot as plt
import pickle
import nibabel as nib
from monai.apps import DecathlonDataset
from sklearn.model_selection import train_test_split
from fastMONAI.vision_all import *
from PIL import Image 
import PIL 
import numpy as np 
import numpy.ma as ma
import matplotlib.pyplot as plt
import cv2


In [ ]:
#msk = PILMask.create(lbl_names[5])
#msk.show(figsize=(5,5), alpha=0.6)

In [ ]:
#img.show(figsize=(5,5))
#msk.show(figsize=(5,5), alpha=0.6)
#plt.imshow(image, cmap='gray') 

In [ ]:
top = nib.load('../data/external/IXI-T1/IXI012-HH-1211-T1.nii.gz')
top2 = nib.load('../data/external/IXI-T1/IXI013-HH-1212-T1.nii.gz')
top3 = nib.load('../data/external/IXI-T1/IXI016-Guys-0697-T1.nii.gz')
side = nib.load('../data/external/IXI-T2/IXI012-HH-1211-T2.nii.gz')
DTI = nib.load('../data/external/IXI-DTI/IXI012-HH-1211-DTI-00.nii.gz')
MRA = nib.load('../data/external/IXI-MRA/IXI012-HH-1211-MRA.nii.gz') 
PD = nib.load('../data/external/IXI-PD/IXI012-HH-1211-PD.nii.gz')
top.shape
#Prøvde først å laste inn daten slik

In [ ]:
camvid = DataBlock(blocks=(ImageBlock, MaskBlock(codes)),
                   get_items=get_image_files,
                   splitter=RandomSplitter(),
                   get_y=get_msk,
                   #batch_tfms=[*aug_transforms(size='half'), Normalize.from_stats(*imagenet_stats)]
                   )
#Eksperimenterte med datablock

In [ ]:
dls = camvid.dataloaders(Path('../data/processed/IXI_2d_slices/Axial'), bs=16)

In [ ]:
dls.show_batch(max_n=4, vmin=1, vmax=30, figsize=(14,10))

In [ ]:
dls.vocab = codes

In [ ]:
name2id = {v:k for k,v in enumerate(codes)}
name2id

In [ ]:
void_code = name2id['Unknown']

In [ ]:
def acc_camvid(inp, targ):
  targ = targ.squeeze(1)
  mask = targ != void_code
  return (inp.argmax(dim=1)[mask]==targ[mask]).float().mean()

In [ ]:
opt = ranger

In [ ]:
learn = unet_learner(dls, resnet34, metrics=acc_camvid, self_attention=True, act_cls=Mish, opt_func=opt)

In [ ]:
image = cv2.imread('../data/processed/IXI_2d_slices/Axial/IXI002-Guys-0828-2d_axial.png')
image_mask = cv2.imread('../data/processed/IXI_2d_slices/AxialMask/IXI002-Guys-0828-2d_axial-mask.png')

#18, 18 ,18 Left Hippocampus

lower = np.array([17,17,17])
upper = np.array([19,19,19])

Hippocampus = cv2.inRange(image_mask, lower, upper)

masked = cv2.bitwise_and(image,image, mask=Hippocampus)

result = image - masked

#mask
#plt.imshow(result, alpha=1)
plt.imshow(image, cmap='gray') # I would add interpolation='none'
plt.imshow(masked, cmap='jet', alpha=0.7) # interpolation='none'

Load_test = nib.load('../data/external/T1/BRAINIX_DICOM_T1_IM-0001-0002.dcm')
temp = plt.imshow(Load_test.get_fdata()[:,128,:])
temp
mappen = "../data/external/IXI-T1"
path = Path(mappen)
path.mkdir(exist_ok=True)

alle_filene = [f for f in os.listdir(path) if f.endswith(".gz")]

i = 0


for x in alle_filene:
    print(x)
    img = plt.imshow(nib.load(mappen + "/" + x).get_fdata()[:,128,:])
    bilde = Image.open(img)
    bilde = bilde.save("{}.png")
    i = i + 1
   
img = top3.get_fdata()[:,128,:]
bilde = plt.imshow(img)
bilde
plt.imshow(img)
plt.imshow(top.get_fdata()[:,120,:])
plt.imshow(side.get_fdata()[:,100,:])
plt.imshow(DTI.get_fdata()[:,90,:])
plt.imshow(MRA.get_fdata()[:,100,:])
plt.imshow(PD.get_fdata()[:,120,:])